# CP Decomposition using ALS

In [41]:
import numpy as np
import tensorly as tl # Used for verification
import matplotlib.pyplot as plt
import torch

## Khatri-Rao Product

In [42]:
"""
Input: A: Array of matrices whose Khatri-Rao product is to be calculated
Output: B: Khatri-Rao product of the matrices in A
"""
def khatri_rao_product(A):
    N = len(A)
    num_cols = A[0].shape[1]
    x_dim = 1
    for i in range(N):
        x_dim *= A[i].shape[0]
    B = np.empty((x_dim, num_cols))

    for i in range(num_cols):
        for j in range(N):
            if j == 0:
                kron_prod = A[j][:, i]
            else:
                kron_prod = np.kron(kron_prod, A[j][:, i])
        B[:, i] = kron_prod

    return B

## Test Khatri Rao Product

In [43]:
"""Khatri-Rao product using tensorly"""
print("Khatri-Rao product using tensorly")
A = [np.random.rand(2, 3), np.random.rand(1, 3), np.random.rand(3, 3)]
Result = tl.tenalg.khatri_rao(A)
print("Result Shape: ", Result.shape)
print("Result:")
print(Result)

"""Khatri Rao product using khatri_rao_product()"""
print("\nKhatri Rao product using khatri_rao_product()")
B = khatri_rao_product(A)
print("Result Shape: ", B.shape)
print("Result:")
print(B)

Khatri-Rao product using tensorly
Result Shape:  (6, 3)
Result:
[[0.18077025 0.03330373 0.00823509]
 [0.06889929 0.13304837 0.00728177]
 [0.22213696 0.02361541 0.01059998]
 [0.13699133 0.02812971 0.21541803]
 [0.05221327 0.11237817 0.19048053]
 [0.16833985 0.01994655 0.27728027]]

Khatri Rao product using khatri_rao_product()
Result Shape:  (6, 3)
Result:
[[0.18077025 0.03330373 0.00823509]
 [0.06889929 0.13304837 0.00728177]
 [0.22213696 0.02361541 0.01059998]
 [0.13699133 0.02812971 0.21541803]
 [0.05221327 0.11237817 0.19048053]
 [0.16833985 0.01994655 0.27728027]]


## Mode-N Matricization

In [44]:
"""
Input:  1) tensor: Input tensor
        2) n: mode along which to matricize the tensor (mode is 0-indexed)
Output: matrix: n-mode matricization of the tensor
"""
# Take mode = 0 for the first mode, mode = 1 for the second mode, ...., mode = n-1 for nth mode
def mode_n_matricization(tensor, n):
    mode = n
    # Get the size of the original tensor
    sz = tensor.shape
    # print(f"tensor size: {sz}")

    # Permute the dimensions of the tensor to bring the chosen mode to the front
    # This will make it easy to reshape the tensor into a matrix along the chosen mode
    permuted_dimensions = list(range(len(sz)))
#     print('Before Permuation, dimensions: ', permuted_dimensions)
    permuted_dimensions.remove(mode)
    permuted_dimensions.insert(0, mode)
#     print('After Permuation, dimensions: ', permuted_dimensions)
    permuted_tensor = tensor.transpose(*permuted_dimensions)
#     print(f"permuted tensor size: {permuted_tensor.size()}")

    # Reshape the permuted tensor into a matrix along the chosen mode
    matrix = permuted_tensor.reshape(sz[mode], -1)
#     print(f"matrix size: {matrix.size()}")

#     print(f"n-mode matricization along mode {mode}:")
#     print(matrix)

    return matrix

## Test Mode-N Matricization

In [45]:
my_tensor = np.empty((2,3,4))
my_tensor[0, :, :] = np.array([[1,4,7,10], [2,5,8,11], [3,6,9,12]])
my_tensor[1, :, :] = np.array([[13,16,19,22], [14,17,20,23], [15,18,21,24]])
print(f"Original Tensor: {my_tensor.shape}")

for n in range(len(my_tensor.shape)):
    print(f"\nn-mode matricization along mode {n}:")
    matrix = mode_n_matricization(my_tensor, n)
    print(matrix)

Original Tensor: (2, 3, 4)

n-mode matricization along mode 0:
[[ 1.  4.  7. 10.  2.  5.  8. 11.  3.  6.  9. 12.]
 [13. 16. 19. 22. 14. 17. 20. 23. 15. 18. 21. 24.]]

n-mode matricization along mode 1:
[[ 1.  4.  7. 10. 13. 16. 19. 22.]
 [ 2.  5.  8. 11. 14. 17. 20. 23.]
 [ 3.  6.  9. 12. 15. 18. 21. 24.]]

n-mode matricization along mode 2:
[[ 1.  2.  3. 13. 14. 15.]
 [ 4.  5.  6. 16. 17. 18.]
 [ 7.  8.  9. 19. 20. 21.]
 [10. 11. 12. 22. 23. 24.]]


## Einsum functions

In [46]:
"""
Input: N: Length of Array of factor matrices
Output: equation: equation string for torch.einsum()
"""
def einsum_equation(N):
    equation = ''    
    for i in range(N):
        if(i != 0):
            equation += ','
        equation += f'{chr(97 + i)}z'

    equation += '->'

    for i in range(N):
        equation += f'{chr(97 + i)}'
    # print(f"equation: {equation}")
    return equation

A = [np.random.rand(2, 3), np.random.rand(1, 3), np.random.rand(3, 3)]
eqn = einsum_equation(3)
print(f"equation: {eqn}")

equation: az,bz,cz->abc


## CP-ALS

In [47]:
"""
Input:  1) T: Tensor of size (s_1, s_2, ..., s_N)
        2) R: CP-Rank of the Tensor
Output: 1) A: Array of N factor matrices of size (s_i, R)
"""
def CP_ALS(T, R, max_iter=100, tol=1e-5):
    N = len(T.shape)
    sz = T.shape
    A = []
    fitness_arr=[]
    for n in range(N):
        An = np.random.rand(sz[n], R)
        A.append(An)

    fitness= 1 - (np.linalg.norm(T - np.einsum(eqn, *A)).item) /np.linalg.norm(T)
    fitness_arr.append(fitness)
    # Iterate until convergence
    for iter in range(max_iter):
        prev_factors = A.copy()
        
        for n in range(N):
            # Calculate the Khatri-Rao product of all the factor matrices except the n-th factor matrix
            B = khatri_rao_product(A[:n] + A[n+1:])
            B_pinv = np.linalg.pinv(B)

            # Calculate the n-mode matricization of the tensor
            Tn = mode_n_matricization(T, n)
            
            # Calculate the n-th factor matrix
            AnT = np.matmul(B_pinv, Tn.T)
            A[n] = AnT.T
          
            fitness= 1 - (np.linalg.norm(T - np.einsum(eqn, *A)).item) /np.linalg.norm(T)
            fitness_arr.append(fitness)
        # Check for convergence
        if all(np.linalg.norm(A[i] - prev_factors[i]) < tol for i in range(T.ndim)):
            print(f"Converged at iteration {iter}")
            break

    return A,fitness_arr
    

## Test CP-ALS

In [48]:
"""CP Decomposition using tensorly"""
print("CP Decomposition using tensorly")
tensor = np.random.rand(3, 4, 5, 6)
rank = 3

print(f"Rank: {rank}")
print(f"Tensor: {tensor.shape}")
print('Original Tensor:')
print(tensor)

cp_decomp = tl.decomposition.CP(rank)
weights, factor_matrices = cp_decomp.fit_transform(tensor)

print('\nFactor Matrices:')
for i in range(len(factor_matrices)):
    print("\nFactor Matrix {} Shape: {}".format(i+1, factor_matrices[i].shape))
    print("Factor Matrix {}: ".format(i+1))
    print(factor_matrices[i])

reconstructed_tensor = tl.cp_tensor.cp_to_tensor((weights, factor_matrices))

# Norm difference between original tensor and reconstructed tensor
print('\nNorm difference between original tensor and reconstructed tensor: ', np.linalg.norm(tensor - reconstructed_tensor).item())


"""CP Decomposition using CP_ALS()"""
print("\nCP Decomposition using CP_ALS()")
A ,fitness_arr= CP_ALS(tensor, rank)

print('Factor Matrices:')
for i in range(len(A)):
    print("\nFactor Matrix {} Shape: {}".format(i+1, A[i].shape))
    print("Factor Matrix {}: ".format(i+1))
    print(A[i])

# Norm difference between original tensor and reconstructed tensor
eqn = einsum_equation(len(A))
print('\nNorm difference between original tensor and reconstructed tensor: ', np.linalg.norm(tensor - np.einsum(eqn, *A)).item())

CP Decomposition using tensorly
Rank: 3
Tensor: (3, 4, 5, 6)
Original Tensor:
[[[[4.80008635e-01 2.95925449e-01 2.93202522e-02 8.93871104e-01
    9.39838459e-01 2.50712660e-01]
   [7.19879376e-01 7.14961681e-01 7.25886158e-01 5.31526880e-01
    5.24545203e-01 4.75809652e-01]
   [4.78988773e-01 9.17442354e-01 7.28709530e-01 5.22086506e-01
    1.42214601e-01 7.99533577e-01]
   [6.76897720e-01 6.12269326e-01 2.24937905e-01 6.36212814e-01
    3.00017411e-01 3.83130963e-01]
   [7.89551152e-02 2.26684554e-01 7.70876560e-01 5.33458056e-02
    9.45030156e-01 6.60073097e-01]]

  [[5.17575401e-01 8.74656452e-01 3.69740739e-03 7.69221002e-01
    2.08297679e-01 1.46207443e-01]
   [7.37533082e-01 4.79488724e-02 2.25018038e-01 3.63151355e-01
    5.47405233e-01 4.71969420e-01]
   [2.91322324e-01 5.60126567e-01 8.34925467e-01 7.95147306e-01
    2.23591962e-01 1.32219429e-01]
   [8.59744119e-01 1.36397160e-01 4.38602984e-01 9.89423742e-01
    6.27073664e-01 9.57574410e-01]
   [2.93273694e-01 3.87331058

ValueError: fewer operands provided to einstein sum function than specified in the subscripts string

In [ ]:
#Now plot
T= np.random.randn(5,5,5,5)
print(T)
ranks=[1,2,4,3]
A, fitness_arr = CP_ALS(T, ranks)
itera=[i for i in range(len(fitness_arr))]
plt.plot(itera, fitness_arr)
plt.xlabel('Iteration')
plt.ylabel('Fitness')
plt.title('HOOI tensor1')
plt.show()
